In [2]:
import pickle
import glob
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ovito

In [3]:
path2atoms = "/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/surface_atoms"
atoms_files = glob.glob(os.path.join(path2atoms, "*.pkl"))
print(len(atoms_files))

160


In [5]:
for atom_f in atoms_files:
    with open(atom_f, mode="rb") as f:
        atoms = pickle.load(f)
    break

/Users/y1u0d2/opt/anaconda3/envs/mlp/lib/python3.11/site-packages/ovito/_extensions/averaging.py:2: UserWarning: Did you accidentally install the OVITO package from the PyPI repository in an Anaconda/Miniconda Python interpreter using the 'pip' command? This will likely lead to conflicts with existing libraries in the Anaconda environment, and import of the OVITO module may fail with an error related to the Qt framework. To fix this, please uninstall the ovito pip package by running 'pip uninstall -y ovito PySide6' and then install the OVITO Anaconda package provided by OVITO GmbH. Visit https://docs.ovito.org/python/introduction/installation.html for further instructions. If you would rather like to ignore this warning message, add the following code to the top of your Python script:

  import warnings
  warnings.filterwarnings('ignore', message='.*OVITO.*PyPI')

  import ovito._extensions.pyscript


In [10]:
print("Number of atoms in the cell:" ,len(atoms))

Number of atoms in the cell: 95


In [12]:
for vec in atoms.get_cell():
    print(vec)

[10.  0.  0.]
[ 0. 10.  0.]
[ 0.     0.    29.325]


In [18]:
for coord in atoms.get_scaled_positions():
    print("Si "," ".join([str(i) for i in coord]))

Si  0.9578153475000001 0.012009721049999998 0.3307999488491047
Si  0.22412718675 0.6092826525 0.2822725063938616
Si  0.3149321265 0.11840508675 0.10733947826086908
Si  0.9305825400000001 0.259546941 0.062269552429667474
Si  0.8214232499999998 0.38572154324999997 0.010223488491048335
Si  0.5170446000000001 0.6204535199999999 0.006440079283887185
Si  0.9309897899999999 0.792011655 0.053190388746802444
Si  0.21650143049999998 0.8885991374999999 0.009963792838874334
Si  0.5838498899999999 0.09387479025 0.017245455242966678
Si  0.29053744424999994 0.36184569749999995 0.01617612020460314
Si  0.7671001725000001 0.8989840124999999 0.010513736572889386
Si  0.5547233699999999 0.37178219025000003 0.10168218670076688
Si  0.3798982755 0.7600303125000001 0.04799138363171321
Si  0.11168749799999998 0.04808685825 0.08761279283887473
Si  0.24513599249999998 0.615802725 0.09766454219948817
Si  0.8318569949999999 0.08393992649999998 0.10249691815856705
Si  0.11191759424999999 0.4789952324999999 0.0570960

In [ ]:
# Create QE file from given template and atoms
def create_qe_input(template, atoms):
    pass

In [71]:
path2template = "/Users/y1u0d2/desktop/Lab/result/qe/Si/slab/extrapolate_atoms/template"
with open(os.path.join(path2template, "scf.in"), mode="r") as f:
    lines = [s.strip() for s in f.readlines()]

In [72]:
n_atoms = len(atoms)
cell = atoms.get_cell()
cell = np.array([vec for vec in cell])
scaled_coord = atoms.get_scaled_positions()

In [74]:
for i, l in enumerate(lines):
    if "nat" in l:
        lines[i] = f"nat = {n_atoms}"
    
    if "CELL_PARAMETERS" in l:
        print(i)
        # insert cell block to i+1
        for j, vec in enumerate(cell_block):
            lines.insert(i+j+1, vec)
    
    if "ATOMIC_POSITIONS" in l:
        print(i)
        # insert coord block to i+1
        for j, coord in enumerate(coord_block):
            lines.insert(i+j+1, coord)

# export lines
with open(os.path.join(path2template, "scf_modified.in"), mode="w") as f:
    for l in lines:
        f.write(l+"\n")

29
33


In [75]:
lines

['#************************************************************************************',
 '#*                   Generated by cif2cell 2.0.0 2022-06-14 9:46                    *',
 '#*  T. Bjorkman, Comp. Phys. Commun. 182, 1183-1186 (2011). Please cite generously. *',
 '#*                                                                                  *',
 '#*                              Data obtained from COD                              *',
 '#*                                     Si   ()                                      *',
 '#*             Failed to get author information, No journal information             *',
 '#************************************************************************************',
 '&CONTROL',
 "calculation = 'scf'",
 "restart_mode = 'from_scratch'",
 "prefix = 'qa'",
 "outdir = '/home/y1u0d2/result/qe/Si/wip'",
 "pseudo_dir = '/home/y1u0d2/result/qe/Si/pseudo'",
 'tprnfor = true',
 '/',
 '&SYSTEM',
 'ibrav = 0',
 'nat = 95',
 'ntyp = 1',
 "occupations = '

In [4]:
# Create QE file from given template and atoms
def create_qe_input(atoms, path2template, path2save):
    with open(os.path.join(path2template, "scf.in"), mode="r") as f:
        lines = [s.strip() for s in f.readlines()]
    n_atoms = len(atoms)
    cell = atoms.get_cell()
    cell = np.array([vec for vec in cell])
    scaled_coord = atoms.get_scaled_positions()

    cell_block = []
    for i, vec in enumerate(cell):
        cell_block.append(" ".join([str(i) for i in vec]))

    coord_block = []
    for j, coord in enumerate(scaled_coord):
        coord_block.append(" ".join(["Si"] + [str(i) for i in coord]))

    # modify input
    for i, l in enumerate(lines):
        if "nat" in l:
            lines[i] = f"nat = {n_atoms}"
        
        if "CELL_PARAMETERS" in l:
            for j, vec in enumerate(cell_block):
                lines.insert(i+j+1, vec)
        
        if "ATOMIC_POSITIONS" in l:
            for j, coord in enumerate(coord_block):
                lines.insert(i+j+1, coord)

    # export lines
    with open(os.path.join(path2save, "scf.in"), mode="w") as f:
        for l in lines:
            f.write(l+"\n")

In [6]:
import subprocess
import os
from glob import glob

path2root = '/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/ase_atoms/20230611/qe_issp'
path2template = '/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/ase_atoms/20230611/template'
path2atoms = '/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/ase_atoms/20230611/divided'

atoms_files = glob(os.path.join(path2atoms, "*.pkl"))
for i, atom_f in enumerate(atoms_files):
    # create directory
    path2save = os.path.join(path2root, f"ex{i}")
    if os.path.exists(path2save):
        raise Exception("Directory already exists")
    else:
        os.mkdir(path2save)

    # load atoms
    with open(atom_f, mode="rb") as f:
        atoms = pickle.load(f)
    
    # create qe input
    create_qe_input(atoms, path2template, path2save)

In [88]:
path2target = "/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/surface_atoms/divided/0"
atoms_dirs = glob(os.path.join(path2target, "ex*"))
print(atoms_dirs[:5])

['/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/surface_atoms/divided/0/ex12', '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/surface_atoms/divided/0/ex15', '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/surface_atoms/divided/0/ex23', '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/surface_atoms/divided/0/ex24', '/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/surface_atoms/divided/0/ex25']


In [ ]:
for i, atom_dir in enumerate(atoms_dirs):
    # run qe
    try:
        process = subprocess.Popen(
            f'mpiexec.hydra -n {n_parallel} -machine $TMPDIR/machines pw.x -in {atom_dir}/scf.in > {atom_dir}/scf.out',
            shell=True)
        process.wait()
    except:
        continue

In [79]:
path2atoms = "/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/ml/depth15/0deg_1000eV/surface_atoms"
atoms_files = glob.glob(os.path.join(path2atoms, "*.pkl"))
print(len(atoms_files))

160


In [82]:
# divide atoms_files into 5 and move to each directory
import shutil
from glob import glob

path2atoms = "/home/y1u0d2/result/qe/Si/amorphous/qe"
path2root = '/home/y1u0d2/result/qe/Si/amorphous/20230611'

atoms_dirs = glob(os.path.join(path2atoms, "ex*"))
print(len(atoms_dirs))

n_divide = len(atoms_dirs) // 5
for i in range(5):
    path2save = os.path.join(path2root, f"amorphous_atoms_{i}")
    os.makedirs(path2save, exist_ok=True)
    for atom_d in atoms_dirs[i*n_divide:(i+1)*n_divide]:
        # copy directory
        shutil.copytree(atom_d, os.path.join(path2save, os.path.basename(atom_d)))

In [8]:
path2target = "/home/y1u0d2/result/qe/Si/amorphous/20230611/amorphous_atoms_0"
atoms_dirs = glob(os.path.join(path2target, "ex*"))
n_parallel = 40

for i, atom_dir in enumerate(atoms_dirs):
    # run qe
    try:
        process = subprocess.Popen(
            f'mpiexec.hydra -n {n_parallel} -machine $TMPDIR/machines pw.x -in {atom_dir}/scf.in > {atom_dir}/scf.out',
            shell=True)
        process.wait()
    except:
        continue